In [1]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-124346
Azure region: southcentralus
Subscription id: 25e9e549-bc6b-4c63-bae8-181a9aa56fa7
Resource group: aml-quickstarts-124346


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Compute target exists')
except ComputeTargetException:
    print('Creating a new compute target')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Compute target exists
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-11-03T13:31:00.217000+00:00', 'errors': None, 'creationTime': '2020-11-03T10:24:31.177245+00:00', 'modifiedTime': '2020-11-03T10:24:46.584428+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": choice(0.001, 0.01, 0.1, 1, 10, 100, 1000),
    "--max_iter": choice(range(80,150))
})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 3, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
shutil.copy('train.py', "./training")

est = SKLearn(
    source_directory = "./",
    entry_script = "train.py",
    compute_target = compute_target
)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                    hyperparameter_sampling = ps,
                                    policy = policy,
                                    primary_metric_name = "Accuracy",
                                    primary_metric_goal =PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs = 4,
                                    max_concurrent_runs = 4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
#exp = experiment(ws, name="SKLearn_logistic_regression_experiment")
hyperdrive_run = exp.submit(hyperdrive_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [5]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_05619965-7ac2-413f-b0f1-a6237a966bf6
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_05619965-7ac2-413f-b0f1-a6237a966bf6?wsid=/subscriptions/25e9e549-bc6b-4c63-bae8-181a9aa56fa7/resourcegroups/aml-quickstarts-124346/workspaces/quick-starts-ws-124346

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-03T13:36:03.568855][API][INFO]Experiment created<END>\n"<START>[2020-11-03T13:36:04.6656650Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-11-03T13:36:04.331976][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-03T13:36:04.682797][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_05619965-7ac2-413f-b0f1-a6237a966bf6
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_05619965-7ac2-413f-b0f1-a6237a966bf6?wsid=/subscriptions/25e9e

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User errors were found in at least one of the child runs.",
        "details": [],
        "debugInfo": {
            "message": "[(https://southcentralus.experiments.azureml.net/subscriptions/25e9e549-bc6b-4c63-bae8-181a9aa56fa7/resourceGroups/aml-quickstarts-124346/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-124346/experiments/**SCRUBBED**/runs/HD_05619965-7ac2-413f-b0f1-a6237a966bf6_0, {'error': {\n    \"additional_properties\": {\n        \"componentName\": null\n    },\n    \"error\": {\n        \"additional_properties\": {\n            \"severity\": null\n        },\n        \"code\": \"UserError\",\n        \"message\": \"User program failed with NameError: name 'clean_data' is not defined\",\n        \"details_uri\": \"https://aka.ms/azureml-known-errors\",\n        \"target\": null,\n        \"details\": [],\n

In [6]:
assert(hyperdrive_run.get_status() == "Completed")

AssertionError: 

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
best_run_metrics = best_run.get_metrics()
print('\n Accuracy:', best_run_metrics['Accuracy'])

In [ ]:
model = best_run.register_model(model_name='udacity_project_sklearn_logistic_regression', model_path='outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

# Split into train and test
from sklearn.model_selection import train_test_split
import pandas as pd
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, shuffle=True)

df_train = pd.concat([x_train, y_train], axis=1)

In [ ]:
df_train.head()

In [ ]:
df_train.to_csv('training/training_data.csv')

In [ ]:
exp = Experiment(workspace=ws, name="AutoML-test")

In [ ]:
datastore = ws.get_default_datastore()
datastore

In [ ]:
datastore.upload(src_dir='training/', target_path='data/')

In [ ]:
train_dataset = Dataset.Tabular.from_delimited_files(path=[(dtaastore, ('data/training_data.csv'))])

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_dataset,
    label_column_name="y",
    n_cross_validations=5,
    compute_target = compute_target,
    iterations = 40,
    max_concurrent_iterations = 6,
    max_cores_per_iteration = -1)

In [ ]:
# Submit your automl run
run = exp.submit(config=automl_config, show_output=True)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.
best_run, best_model = run.get_output()

In [ ]:
run.register_model(model_name = "automl_udacity_project", model_path = './outputs/')
print(best_model._final_estimator)

In [ ]:
compute_target_delete()